In [1]:
import bw2data as bd

bd.projects.set_current("multi-level-tds")

In [10]:
biosphere = bd.Database("biosphere")
biosphere.write(
    {
        ("biosphere", "CO2"): {
            "type": "emission",
            "name": "carbon dioxide",
        },
    }
)

if "foreground" in bd.databases:
    del bd.databases["foreground"] # to make sure we create the foreground from scratch
foreground = bd.Database("foreground")
foreground.register()

if "background_2020" in bd.databases:
    del bd.databases["background_2020"] # to make sure we create the foreground from scratch
background_2020 = bd.Database("background_2020")
background_2020.register()

if "background_2030" in bd.databases:
    del bd.databases["background_2030"] # to make sure we create the foreground from scratch
background_2030 = bd.Database("background_2030")
background_2030.register()

100%|██████████| 1/1 [00:00<00:00, 4466.78it/s]

Vacuuming database 


In [11]:
node_co2 = biosphere.get("CO2")

fu = foreground.new_node("fu", name="fu", unit="unit")
fu['reference product'] = "fu"
fu.save()

used_car = foreground.new_node("used_car", name="used_car", unit="unit")
used_car['reference product'] = "used_car"
used_car.save()

car_treatment_2020 = background_2020.new_node("car_treatment", name="car_treatment", unit="car_treatment")
car_treatment_2020['reference product'] = "car_treatment"
car_treatment_2020.save()

# car_treatment_2030 = background_2030.new_node("car_treatment", name="car_treatment", unit="car_treatment")
# car_treatment_2030['reference product'] = "car_treatment"
# car_treatment_2030.save()


In [12]:
fu.new_edge(input=fu, amount=1, type="production").save()
fu.new_edge(input=used_car, amount=-1, type="technosphere").save()

used_car.new_edge(input=used_car, amount=-1, type="production").save()
used_car.new_edge(input=car_treatment_2020, amount=-3, type="technosphere").save()

car_treatment_2020.new_edge(input=car_treatment_2020, amount=-1, type="production").save()
car_treatment_2020.new_edge(input=node_co2, amount=1, type="biosphere").save()

# background_process_2030.new_edge(input=background_process_2030, amount=1, type="production").save()
# background_process_2030.new_edge(input=node_co2, amount=1, type="biosphere").save()


In [17]:
method = ("GWP", "example")
bd.Method(method).write(
    [
        (("biosphere", "CO2"), 1),
    ]
)

In [ ]:
import bw2calc as bc

In [18]:
lca = bc.LCA({fu: 1}, ("GWP", "example"))
lca.lci()
lca.lcia()
lca.score

3.0

## LCA using `bw_timex`


As usual, we need to select a method:

In [19]:
from datetime import datetime

database_dates = {
    "background_2020": datetime.strptime("2020", "%Y"),
    # "background_2030": datetime.strptime("2030", "%Y"),
    "foreground": "dynamic", # flag databases that should be temporally distributed with "dynamic"
}

In [20]:
from bw_timex import TimexLCA

tlca = TimexLCA({fu: 1}, method, database_dates)

In [21]:
tlca.build_timeline()

Starting graph traversal
Calculation count: 2


/Users/timodiepers/Documents/Coding/bw_timex/bw_timex/timex_lca.py:194: UserWarning: No edge filter function provided. Skipping all edges within background databases.
  warnings.warn(
/Users/timodiepers/Documents/Coding/bw_timex/bw_timex/timeline_builder.py:484: Warning: Reference date 2024-01-01 00:00:00 is higher than all provided dates. Data will be taken from the closest lower year.
  warnings.warn(


,date_producer,producer_name,date_consumer,consumer_name,amount,temporal_market_shares
0,2024-01-01,fu,2024-01-01,-1,1.0,None
1,2024-01-01,used_car,2024-01-01,fu,-1.0,None
2,2024-01-01,car_treatment,2024-01-01,used_car,3.0,{'background_2020': 1}


In [22]:
tlca.lci()

/Users/timodiepers/anaconda3/envs/timex/lib/python3.10/site-packages/bw2calc/lca_base.py:127: SparseEfficiencyWarning: splu converted its input to CSC format
  self.solver = factorized(self.technosphere_matrix)


In [23]:
tlca.static_lcia()
tlca.static_score

-3.0000000000000004

In [79]:
tlca.base_lca.score

840.0000000000001

In [76]:
tlca.activity_time_mapping

{(('foreground', 'fu'), 'dynamic'): 187,
 (('foreground', 'first_level_input'), 'dynamic'): 188,
 (('background_2020', 'background_process'), 2020): 189,
 (('background_2030', 'background_process'), 2030): 190,
 (('temporalized', 'background_process'), 2021): 191,
 (('temporalized', 'background_process'), 2022): 192,
 (('temporalized', 'fu'), 2024): 193,
 (('temporalized', 'first_level_input'), 2024): 194}

In [77]:
import pandas as pd

expected_inventory = pd.DataFrame(
    data={
        "date": pd.Series(
            data=[
                "01-01-2019",
                "01-01-2020",
                "01-01-2021",
            ],
            dtype="datetime64[s]",
        ),
        "amount": pd.Series(
            data=[0.2*336, 0.8*336+0.2*504, 0.8*504], dtype="float64"
        ),
        "flow": pd.Series(data=[41, 41, 41], dtype="int"),
        "activity": pd.Series(data=[46, 44, 47], dtype="int"),
    }
)
expected_inventory

,date,amount,flow,activity
0,2019-01-01,67.2,41,46
1,2020-01-01,369.6,41,44
2,2021-01-01,403.2,41,47


In [49]:
tlca.create_labelled_technosphere_dataframe()

,"((foreground, fu), dynamic)","((foreground, first_level_input), dynamic)","((background_2020, background_process), 2020)","((background_2030, background_process), 2030)","((temporalized, background_process), 2019)","((temporalized, background_process), 2020)","((temporalized, background_process), 2021)","((temporalized, first_level_input), 2022)","((temporalized, first_level_input), 2023)","((temporalized, fu), 2024)"
"((foreground, fu), dynamic)",1.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
"((foreground, first_level_input), dynamic)",-1.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
"((background_2020, background_process), 2020)",0.0,-840.0,1.0,0.0,-1.0,-1.0,-0.899808,0.0,0.0,0.0
"((background_2030, background_process), 2030)",0.0,0.0,0.0,1.0,0.0,0.0,-0.100192,0.0,0.0,0.0
"((temporalized, background_process), 2019)",0.0,0.0,0.0,0.0,1.0,0.0,0.000000,-336.0,0.0,0.0
"((temporalized, background_process), 2020)",0.0,0.0,0.0,0.0,0.0,1.0,0.000000,-504.0,-336.0,0.0
"((temporalized, background_process), 2021)",0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,-504.0,0.0
"((temporalized, first_level_input), 2022)",0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.0,0.0,-0.2
"((temporalized, first_level_input), 2023)",0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,1.0,-0.8
"((temporalized, fu), 2024)",0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,1.0


In [50]:
tlca.create_labelled_dynamic_biosphere_dataframe()

,"((foreground, fu), dynamic)","((foreground, first_level_input), dynamic)","((background_2020, background_process), 2020)","((background_2030, background_process), 2030)","((temporalized, background_process), 2019)","((temporalized, background_process), 2020)","((temporalized, background_process), 2021)","((temporalized, first_level_input), 2022)","((temporalized, first_level_input), 2023)","((temporalized, fu), 2024)"
"(181, 2019-01-01T00:00:00)",0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
"(181, 2020-01-01T00:00:00)",0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
"(181, 2021-01-01T00:00:00)",0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [51]:
from dynamic_characterization.ipcc_ar6 import characterize_co2

In [53]:
tlca.dynamic_lcia(metric="GWP", characterization_functions={node_co2.id: characterize_co2})

/Users/timodiepers/anaconda3/envs/timex/lib/python3.10/site-packages/dynamic_characterization/dynamic_characterization.py:262: UserWarning: Using bw_timex's default CO2 characterization function for GWP reference.
  warnings.warn(


,date,amount,flow,activity
0,2019-01-01,67.2,181,186
1,2020-01-01,369.6,181,187
2,2021-01-01,403.2,181,188


In [54]:
tlca.dynamic_score

840.0

In [55]:
tlca.base_lca.score

840.0000000000001

In [56]:
tlca.static_lcia()
tlca.static_score

840.0